In [ ]:
import pandas as pd
import numpy as np

# Make numpcy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import matplotlib.pyplot as plt
import matplotlib as mat
import colordict as cd
import seaborn as sns
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from flowability_data_upload.Research import main
from flowability_data_upload.Research.main import trymodel, makemodel, modeltest
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.graphics.regressionplots import abline_plot
import math

In [3]:
cdd = []
icdd = []
cdd = ["Id", "Img Id", "Powder", "Filter0", "Filter1", "Filter2", "Filter3", "Filter4", "Filter5", "Filter6"] #general columns to drop
icdd = cdd.copy()
icdd.remove("Powder")

In [4]:
x = Data()
x.importnewpowders()
data = x.data.copy() 
datalist = x.datalist.copy()

# powders with flow
flow = x.flow.copy()
# powders with no flow
noflow = x.noflow.copy()
flowlist = x.flowpowders.copy()
noflowlist = x.noflowpowders.copy()
alldatalist = x.alldatalist.copy()

           Da      Dp  FWidth  FLength  ELength  EWidth     Volume     Area  \
0      31.076  34.663  27.644   35.973   34.802  27.871  15712.926  758.453   
1      27.162  29.344  27.434   28.583   27.432  26.912  10492.573  579.445   
2      27.239  29.955  27.099   29.289   28.443  26.148  10582.306  582.744   
3      33.260  38.347  27.040   44.811   45.210  25.836  19265.652  868.854   
4      28.584  30.520  26.918   31.386   30.804  26.592  12228.183  641.700   
...       ...     ...     ...      ...      ...     ...        ...      ...   
11313   3.482   3.924   3.218    4.275    3.876   2.981     22.097    9.520   
11314   3.459   4.264   3.532    4.816    4.121   3.167     21.665    9.396   
11315   3.403   3.871   3.314    4.106    3.789   2.953     20.627    9.093   
11316   3.398   4.532   3.888    4.532    3.881   3.094     20.549    9.070   
11317   3.343   3.997   3.378    4.466    3.924   2.931     19.567    8.779   

       Perimeter  CHull  Area  ...  Transparency  C

In [6]:
# Powder Info
# 0: no flow, 1: bad flow, 2: good flow
plist = []

for powder in alldatalist:
    plist.append([powder["Powder"][0], len(powder.index), powder["Flow"][0], powder["Flow Class"][0]])
powderinfo = pd.DataFrame(plist, columns=["Powder", "# of Particles", "Flow", "Flow Class"])
powderinfo

,Powder,# of Particles,Flow,Flow Class
0,S1VRC0706-3_Excel.txt,803,20.013333,1
1,WIP-C1.txt,13204,21.786667,1
2,WIP-W1.txt,14877,25.786667,1
3,MW_Ti-Nb-Zr_0-63.txt,8462,33.970000,1
4,WIP-W2.txt,33839,34.310000,1
5,MW_Ti-Nb-Zr_64-150.txt,586,26.100000,1
6,MW_I_718_0-10.txt,79935,0.000000,0
7,S1VRC0706-5_Excel.txt,920,0.000000,0
8,V5_Al-P1015_99.8%-Al.txt,303,53.300000,1
9,MW_Ti-Nb_64-150.txt,684,2.380000,1


In [7]:
pflist = []
pdflist = []
fplist = []
fpdlist = []
flist = []
idlist = []
pdict = {}
for i in data.columns:
    pflist.append([])
for powder in alldatalist:
    print(powder)
    powder = powder.drop(columns = icdd)
    pid = powder["Powder"][0]
    pf = powder.drop(columns = ["Powder", "Flow", "Flow Class"])
    idlist.append(pid)
    
    #print("Powder Id: ", powder["Id"][0])
    count = 0
    feat = []
    index = []
    for f in pf:
        ilist = [powder["Powder"][0]]
        #jlist = [f]
        index.append(f)
        #print(f, ": ") #feature
        print(f)
        ft = pf[f].to_numpy()
        mean = ft.mean()
        med = np.median(ft)
        fmax = np.amax(ft)
        fmin = np.amin(ft)
        std = ft.std()
        size = ft.size
        zeros = size - np.count_nonzero(ft)
        #print(mean, "\n", med, "\n", std)
        jlist = [mean, med, std, fmax, fmin, zeros]
        flist = ["Mean", "Median", "Std", "Max", "Min", "# 0's"]
        ilist.extend(jlist)
        
        #print(ilist)
        #print(pflist)
        pflist[count].append(jlist)
        fplist.append
        feat.append(jlist)
        count = count + 1
    fplist.append(feat)
    df = pd.DataFrame(feat, columns=flist, index = index)
    for i in ["Mean", "Median", "Max", "Min", "Std"]:
        df[i] = df[i].round(4)
        df[i] = df[i].apply(lambda x: '%.5f' % x) 
    pdict[pid] = df
    fpdlist.append(df)
    
for i in pflist:
    df = pd.DataFrame(i, columns=flist, index = idlist)
    #for i in ["Mean", "Median", "Max", "Min", "Std"]:
        #df[i].round(4)
        #df[i] = df[i].apply(lambda x: '%.5f' % x) 
    df = df.round(1)
    pdflist.append(df)

pdfdict = {}
count = 0
for i in data.columns:
    pdfdict[i] = pdflist[count]
    count = count + 1

      Id  Img Id      Da      Dp  FWidth  FLength  ELength  EWidth     Volume  \
0    487     286  31.076  34.663  27.644   35.973   34.802  27.871  15712.926   
1    227     189  27.162  29.344  27.434   28.583   27.432  26.912  10492.573   
2    793     536  27.239  29.955  27.099   29.289   28.443  26.148  10582.306   
3    598     327  33.260  38.347  27.040   44.811   45.210  25.836  19265.652   
4    688     354  28.584  30.520  26.918   31.386   30.804  26.592  12228.183   
..   ...     ...     ...     ...     ...      ...      ...     ...        ...   
798  137     156   4.474   4.900   3.740    5.785    5.352   3.757     46.893   
799  215     184   4.474   4.900   3.740    5.785    5.352   3.757     46.893   
800  632     334   5.285   5.958   3.740    7.364    7.098   3.956     77.298   
801  692     356   3.836   4.997   3.629    5.959    4.607   3.342     29.547   
802  186     174   4.143   6.185   3.397    7.611    6.278   2.914     37.231   

        Area  ...  Filter4 

Da
Dp
FWidth
FLength
ELength
EWidth
Volume
Area
Perimeter
CHull  Area
CHull Perimeter
Sphericity
Compactness
Roundness
Ellipse Ratio
Circularity
Solidity
Concavity
Convexity
Extent
hash
Transparency
Curvature
Surface Area
L/W Ratio
W/L Aspect Ratio
CHull Surface Area
Ellipticity
      Id  Img Id      Da      Dp  FWidth  FLength  ELength  EWidth     Volume  \
0    223     252  44.723  48.830  44.903   46.985   45.346  44.150  46836.890   
1    326     279  47.089  48.919  44.901   50.022   49.450  44.838  54672.672   
2     81     205  44.754  48.572  44.899   46.778   45.529  44.025  46935.508   
3    539     339  45.135  52.216  44.881   48.543   45.900  44.441  48145.238   
4    683     371  45.178  47.920  44.822   47.296   45.912  44.467  48280.096   
..   ...     ...     ...     ...     ...      ...      ...     ...        ...   
915   38     186   6.942   8.967   6.758   10.841    7.973   6.132    175.150   
916  138     224   7.846   9.471   6.690   10.111    9.906   6.283    25

        Id  Img Id      Da       Dp  FWidth  FLength  ELength  EWidth  \
0     6669     409  81.063   85.223  79.944   88.048   81.789  80.710   
1     1149     155  78.077  105.681  44.865  138.111  151.973  42.271   
2      886     143  77.928   90.164  49.040  114.979  119.630  51.181   
3     4452     295  76.424  117.927  56.865  130.320  128.998  53.370   
4     4410     294  72.572   91.934  41.224  119.751  130.362  41.874   
...    ...     ...     ...      ...     ...      ...      ...     ...   
6887   410     115   5.798    6.193   5.378    6.703    6.224   5.298   
6888  6540     395   5.125    5.760   4.666    6.243    5.595   4.786   
6889  6780     422   4.972    5.325   4.387    6.354    5.884   4.046   
6890  3460     255   4.136    4.398   3.687    4.748    4.452   3.661   
6891  5646     344   3.786    4.261   3.565    4.880    4.354   3.190   

          Volume      Area  ...  Filter4  Filter5  Filter6  L/W Ratio  \
0     278907.280  5160.962  ...   Reject   Reject 

Dp
FWidth
FLength
ELength
EWidth
Volume
Area
Perimeter
CHull  Area
CHull Perimeter
Sphericity
Compactness
Roundness
Ellipse Ratio
Circularity
Solidity
Concavity
Convexity
Extent
hash
Transparency
Curvature
Surface Area
L/W Ratio
W/L Aspect Ratio
CHull Surface Area
Ellipticity
       Id  Img Id      Da      Dp  FWidth  FLength  ELength  EWidth  \
0     389     162  74.901  85.625  55.464  102.129  106.901  54.538   
1     399     164  73.253  79.098  63.633   91.108   88.914  61.370   
2     768     194  70.944  78.621  59.528   88.874   87.667  57.917   
3     696     187  70.284  83.418  56.980   98.522  100.336  51.948   
4     936     208  70.109  77.013  56.523   88.606   90.423  55.788   
...   ...     ...     ...     ...     ...      ...      ...     ...   
1427  731     191   9.074  10.397   8.177   12.160   10.759   7.641   
1428  223     151   8.257   9.159   8.127    9.705    8.971   7.684   
1429  497     170   7.929   9.091   7.219   10.476    9.704   6.514   
1430  907    

Da
Dp
FWidth
FLength
ELength
EWidth
Volume
Area
Perimeter
CHull  Area
CHull Perimeter
Sphericity
Compactness
Roundness
Ellipse Ratio
Circularity
Solidity
Concavity
Convexity
Extent
hash
Transparency
Curvature
Surface Area
L/W Ratio
W/L Aspect Ratio
CHull Surface Area
Ellipticity
          Id  Img Id      Da       Dp  FWidth  FLength  ELength  EWidth  \
0       5844     223  80.952  111.635  50.545  145.185  145.717  47.833   
1       5722     221  80.224  123.669  49.696  152.029  154.001  45.285   
2       3034     177  79.852  102.733  53.829  135.408  139.943  48.923   
3      10849     350  76.722  108.419  54.695  126.453  131.013  49.002   
4       4504     201  75.945  115.411  52.223  127.799  119.164  49.596   
...      ...     ...     ...      ...     ...      ...      ...     ...   
11313   1988     155   3.482    3.924   3.218    4.275    3.876   2.981   
11314   4909     208   3.459    4.264   3.532    4.816    4.121   3.167   
11315  11038     363   3.403    3.871   3.314

In [8]:
cols = []
for i in data.columns:
    cols.append(i)
cols.remove("Flow")
cols.remove("Flow Class")

def featurestat(stat):
    plist = []
    for powder in alldatalist:
        tlist = []

        for feature in cols:
            tlist.append(pdict.get(powder["Id"][0])[stat][feature])
        tlist.extend([powder["Flow"][0], powder["Flow Class"][0]])
        plist.append(tlist)
    df = pd.DataFrame(plist, columns = data.columns, index = idlist)
    df = df.astype(float)
    return df

In [9]:
data

,Da,Dp,FWidth,FLength,ELength,EWidth,Volume,Area,Perimeter,CHull Area,...,Transparency,Curvature,Surface Area,L/W Ratio,W/L Aspect Ratio,CHull Surface Area,Ellipticity,Flow,Flow Class,Powder
0,31.076,34.663,27.644,35.973,34.802,27.871,15712.926,758.453,108.897,803.897,...,0.318,0.0,3033.813,1.301,0.768,3215.587,1.249,20.013333,1,S1VRC0706-3_Excel.txt
1,27.162,29.344,27.434,28.583,27.432,26.912,10492.573,579.445,92.187,602.148,...,0.329,0.0,2317.780,1.042,0.960,2408.590,1.019,20.013333,1,S1VRC0706-3_Excel.txt
2,27.239,29.955,27.099,29.289,28.443,26.148,10582.306,582.744,94.107,611.136,...,0.336,0.0,2330.975,1.081,0.925,2444.545,1.088,20.013333,1,S1VRC0706-3_Excel.txt
3,33.260,38.347,27.040,44.811,45.210,25.836,19265.652,868.854,120.471,930.055,...,0.280,0.0,3475.415,1.657,0.603,3720.222,1.750,20.013333,1,S1VRC0706-3_Excel.txt
4,28.584,30.520,26.918,31.386,30.804,26.592,12228.183,641.700,95.881,666.650,...,0.321,0.0,2566.800,1.166,0.858,2666.602,1.158,20.013333,1,S1VRC0706-3_Excel.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11313,3.482,3.924,3.218,4.275,3.876,2.981,22.097,9.520,12.328,10.204,...,0.449,0.0,38.081,1.328,0.753,40.815,1.300,0.000000,0,Valimet_6061_SPE.txt
11314,3.459,4.264,3.532,4.816,4.121,3.167,21.665,9.396,13.395,10.177,...,0.459,0.0,37.583,1.363,0.733,40.708,1.301,0.000000,0,Valimet_6061_SPE.txt
11315,3.403,3.871,3.314,4.106,3.789,2.953,20.627,9.093,12.161,9.871,...,0.450,0.0,36.373,1.239,0.807,39.485,1.283,0.000000,0,Valimet_6061_SPE.txt
11316,3.398,4.532,3.888,4.532,3.881,3.094,20.549,9.070,14.237,11.551,...,0.431,0.0,36.281,1.166,0.858,46.203,1.255,0.000000,0,Valimet_6061_SPE.txt


In [10]:
data.to_pickle('./data.pkl')

In [313]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import matplotlib.pyplot as plt
import matplotlib as mat
import colordict as cd
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Make numpcy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
module_path = module_path + "/flowability_data_upload/PSD_TurboSync"


class Data:

    def __init__(self):
        # these lists are lists of powder specific dataframes
        self.alldatalist = []
        self.datalist = []
        self.flowpowders = []
        self.noflowpowders = []

        # below dataframes are not separated by powder
        self.alldata = pd.DataFrame()  # all of the data
        self.flowall = pd.DataFrame()  # powders with flow
        self.noflowall = pd.DataFrame()  # powders with no flow
        self.data = pd.DataFrame()  # all of the data
        self.flow = pd.DataFrame()  # powders with flow
        self.noflow = pd.DataFrame()  # powders with no flow

    def importpowders(self):
        cdd = []
        icdd = []
        cdd = ["Id", "Img Id", "Filter0", "Filter1", "Filter2", "Filter3", "Filter4", "Filter5",
               "Filter6", "hash"]  # general columns to drop
        icdd = cdd.copy()
        icdd.remove("Id")
        datanamelist = ["July15B.csv", "JulyB1.csv", "JulyB2.csv", "JulyB4.csv", "JulyB5.csv", "JulyB6.csv",
                        "JulyB10.csv",
                        "JulyB12.csv", "JulyB13.csv", "JulyB14.csv", "JulyB16.csv", "JulyB17.csv", "JulyB18.csv",
                        "JulyB19.csv", "JulyB20.csv"]
        for i in datanamelist:
            idata = pd.read_csv(i);
            pdata = idata.drop(columns=cdd)  # powder data minus general unwanted columns

            if idata["Flow"][0] != 0:
                pdata["Flow Class"] = pdata["Flow Class"] * 0 + 1
                idata["Flow Class"] = idata["Flow Class"] * 0 + 1
                self.flowall = self.flowall.append(idata)
                self.flowpowders.append(pdata)
            else:
                self.noflowall = self.noflowall.append(idata)
                self.noflowpowders.append(pdata)
            self.datalist.append(pdata)  # flow class might be broke
            self.alldatalist.append(idata)
            self.alldata = self.alldata.append(idata)

        self.flow = self.flowall.drop(columns=cdd)
        self.noflow = self.noflowall.drop(columns=cdd)
        self.data = self.alldata.drop(columns=cdd)
        print(self.data)
        
        
    def importnewpowders(self):
        cdd = []
        icdd = []
        cdd = ["Id", "Img Id", "Filter0", "Filter1", "Filter2", "Filter3", "Filter4", "Filter5", "Filter6", "hash"]  # general columns to drop
        icdd = cdd.copy()
        icdd.remove("Id")
        
        datanamelist = os.listdir(module_path)
        datanamelist.pop(datanamelist.index('.ipynb_checkpoints'))
        flowvaldf = pd.read_excel(module_path + "/" + datanamelist.pop(datanamelist.index('AATurboSync_Flowability_HF_Repository.xlsx')), index_col=9)
        #print(datanamelist)
        #print(flowvaldf)
        for i in datanamelist:
            #print(i)
            if i[:2] == "~$":
                break
       
            #print(module_path)
            #idata = pd.read_csv(i);
            idata = pd.read_csv(module_path + "/" + i, sep = '\t', header=(0), skiprows = [1], encoding='cp1252')
            #print(idata)
            
            if i[:-4] != "PA_Ni-914-3":
                flowval = flowvaldf.loc[i[:-4], "Average Hall Flow"]
                #print(flowval)
                if flowval == "No Flow":
                    flowval = 0
                    flowcval = 0
                else:
                    flowcval = 1
                    
            
                idata["Flow"] = [flowval] * len(idata.index)
                idata["Flow Class"] = [flowcval] * len(idata.index)
                idata["Powder"] = [i] * len(idata.index)
                if len(idata.columns) > 40:
                    idata = idata.drop(columns = ["Fiber Length", "Fiber Width"])

                #print(idata)
                pdata = idata.drop(columns=cdd)  # powder data minus general unwanted columns
              
                if idata["Flow"][0] != 0:
                    pdata["Flow Class"] = pdata["Flow Class"] * 0 + flowcval
                    idata["Flow Class"] = idata["Flow Class"] * 0 + flowcval
                    self.flowall = self.flowall.append(idata)
                    self.flowpowders.append(pdata)
                else:
                    self.noflowall = self.noflowall.append(idata)
                    self.noflowpowders.append(pdata)
                self.datalist.append(pdata)  # flow class might be broke
                self.alldatalist.append(idata)
                self.alldata = self.alldata.append(idata)

        self.flow = self.flowall.drop(columns=cdd)
        self.noflow = self.noflowall.drop(columns=cdd)
        self.data = self.alldata.drop(columns=cdd)
        print(self.data)

    def coltest(self, cols, ctd=[]):
        plist = []
        for i in self.datalist:
            plist.append(i.sample(n=2842))
        df = pd.concat(plist)
        df = df.sample(frac=1)
        flowdf = df[df["Flow Class"] == 1]
        results = []
        datar = []
        dfr = []
        results.append(datar)
        results.append(dfr)
        for i in ctd:
            if i in cols:
                cols.remove(i)
        for i in cols:
            cttd = ctd.copy()
            cttd.append(i)
            print("Feature dropped: ", i)
            results[0].append([i, "Data", trymodel(df, flowdf, self.data, ctd=cttd)])
            results[1].append([i, "Df", trymodel(df, flowdf, df, ctd=cttd)])
            print("\n\n\n\n\n\n\n\n\n")
        return results

def trymodel(c, r, t, visualize=True,
             ctd=[]):  # returns 1x4 array containing 4 R^2 values. Classification, Regression, Predicted Regression,
    # and Overall
    if len(ctd) != 0:
        c = c.drop(columns=ctd)
        r = r.drop(columns=ctd)
        t = t.drop(columns=ctd)
    def pfix(arr):  # it works (flips axes or something)
        fppp = np.array(arr)
        fppp = np.absolute(fppp)
        fpp = fppp > 0
        newfp = fppp[fpp]
        return newfp
    cmodel = makemodel(c.drop(columns="Flow"), "Flow Class")
    
    cps = cmodel.predict(t.drop(columns=["Flow", "Flow Class"]))  # classify predictions
    cps = pfix(cps)  # fixes cps and rps into 1d arrays of floats, instead of 2d arrays of floats
    cps = np.where(cps < .5, 0,
                   1)  # forces predictions into classification of 1 or 0 (flow or no flow) (to be revised later)
    cexp = t["Flow Class"].to_numpy()  # classify expected values
    print(cps.shape)
    print(cexp.shape)
    cstats = (cps * cexp) * 2
    cstats = cstats + (cexp - cps)  # breaks results into one array of values 0, 1, -1, and 2 (specified below)
    cstats = np.where(cstats < 0, 3, cstats)

    csum = np.bincount(cstats.astype(int), minlength=4)
    tn = csum[0]  # true negative, red
    fn = csum[1]  # false negative, yellow
    fp = csum[3]  # false positive, orange
    tp = csum[2]  # true positive, green

    cp = tn + tp  # correct predictions
    tot = tn + fn + fp + tp

    print("Classify results:")
    print("\nNegative Predictions: ", (tn + fn) / tot * 100, "%\tActual Negative: ", (tn + fp) / tot * 100, "%")
    print("Positive Predicitons: ", (tp + fp) / tot * 100, "%\tActual Positive: ", (tp + fn) / tot * 100, "%")
    print("\nTrue Negatives: ", tn, "\t", tn / tot * 100, "%")
    print("False Negatives: ", fn, "  ", fn / tot * 100, "%")
    print("False Positives: ", fp, "\t", fp / tot * 100, "%")
    print("True Positives: ", tp, "\t", tp / tot * 100, "%")
    print("\nNo Flow Predicitons: ", (tn / (tn + fn)) * 100, "% Correct")
    print("Flow Predictions: ", (tp / (tp + fp)) * 100, "% Correct")
    print("\nCorrect Predictions:", cp, "  ", cp / tot * 100, "%")
    print(confusion_matrix(cexp, cps))

    # rmodel = makemodel(r.drop(columns="Flow Class"))
    # if visualize:
    #     return modeltest(cmodel, rmodel, t)
    return cmodel


In [298]:

def makemodel(df, y="Flow"):
    df_features = df.copy()
    print(df_features.shape)
    df_labels = df_features.pop(y)
    df_features = np.array(df_features)

    normalize = preprocessing.Normalization()

    normalize.adapt(df_features)

    # norm_df_model = tf.keras.Sequential([
    #     normalize,
    #     layers.Dense(64),
    #     layers.Dense(1)
    # ])
    
    norm_df_model = tf.keras.Sequential()

    norm_df_model.add(layers.Conv1D(filters=10, kernel_size=5, padding='same', activation='tanh',
                         input_shape=(df_features.shape[1],1)))
    norm_df_model.add(layers.BatchNormalization())

    norm_df_model.add(layers.MaxPooling1D(pool_size=4))

    norm_df_model.add(layers.LSTM(64))

    norm_df_model.add(layers.Dense(units=df_features.shape[1], activation='tanh'))
    norm_df_model.add(layers.Dense(1, activation='tanh'))

    norm_df_model.compile(optimizer='adam', loss='mean_squared_error')
    
#     model = tf.keras.Sequential()

#     model.add(
#         layers.Conv1D(filters=256, kernel_size=3, strides=1, activation='relu', input_shape=(99, 40), name='block1_conv1'))
#     model.add(layers.MaxPool1D(pool_size=2, name='block1_pool1'))
#     model.add(layers.BatchNormalization(momentum=0.9, epsilon=1e-5, axis=1))

#     model.add(layers.Conv1D(filters=256, kernel_size=3, strides=1, activation='relu', name='block1_conv2'))
#     model.add(layers.MaxPool1D(pool_size=2, name='block1_pool2'))

#     model.add(layers.Flatten(name='block1_flat1'))
#     model.add(layers.Dropout(0.5, name='block1_drop1'))

#     model.add(layers.Dense(512, activation='relu', name='block2_dense2'))
# #     model.add(layers.core.MaxoutDense(512, nb_feature=4, name="block2_maxout2"))
#     model.add(layers.Dropout(0.5, name='block2_drop2'))

#     model.add(layers.Dense(512, activation='relu', name='block2_dense3'))
# #     model.add(layers.MaxoutDense(512, nb_feature=4, name="block2_maxout3"))
#     model.add(layers.Dense(classe_nums, activation='softmax', name="predict"))


#     norm_df_model = tf.keras.Sequential(
#     [   
#         normalize,
#         layers.Conv1D(32, 3, activation='relu',input_shape=input_shape[1:])(x)
#         layers.Dense(64, activation="tanh", name="layer1"),
#         layers.Dense(32, activation="tanh", name="layer2"),
#         layers.Dense(1, name="layer3"),
#     ]
# )

#     norm_df_model.compile(loss=tf.losses.MeanSquaredError(),
#                           optimizer=tf.optimizers.Adam())

    model = tf.keras.Sequential()
        
    # Stage 2
    model.add(layers.Convolution1D(filters=32,  kernel_size=3, padding='same', input_shape=(df_features.shape[1],1)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('sigmoid'))
    model.add(layers.MaxPooling1D(pool_size=2))    
    
    # Stage 3
    model.add(layers.Convolution1D(filters=64,  kernel_size=3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('sigmoid'))
    model.add(layers.MaxPooling1D(pool_size=2))    
        
    # Stage 4
    model.add(layers.Convolution1D(filters=128,  kernel_size=3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('tanh'))
    model.add(layers.MaxPooling1D(pool_size=2))    
    
    model.add(layers.Flatten())
    
    model.add(layers.Dense(256, activation='tanh', name='fc_'+str(1)))
    
    model.add(layers.Dense(128, activation='tanh', name='fc_'+str(2)))
    
    model.add(layers.Dense(1, activation='sigmoid', name='fc_'+str(4)))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    es = tf.keras.callbacks.EarlyStopping(monitor='accuracy', 
                                   mode='max', # don't minimize the accuracy!
                                   patience=10,
                                   restore_best_weights=True)


    model.fit(df_features, df_labels, callbacks=[es], epochs=1)

    model.summary()

    return model

In [311]:
data["Powder"].value_counts()

test1 = data[data['Powder']=='S1VRC0706-2_Excel.txt']
test2 = data[data['Powder']=='MW_I_718_0-10.txt']
test = pd.concat([test1,test2])

train = data[data['Powder']!=('S1VRC0706-2_Excel.txt' or 'MW_I_718_0-10.txt')]

In [295]:
test

,Da,Dp,FWidth,FLength,ELength,EWidth,Volume,Area,Perimeter,CHull Area,...,Transparency,Curvature,Surface Area,L/W Ratio,W/L Aspect Ratio,CHull Surface Area,Ellipticity,Flow,Flow Class,Powder
0,122.762,145.019,122.713,145.928,137.220,116.988,968709.930,11836.409,455.591,12668.265,...,0.259,0.0,47345.635,1.189,0.841,50673.060,1.173,30.153333,1,S1VRC0706-2_Excel.txt
1,107.417,142.438,109.688,123.240,114.467,105.792,648958.184,9062.234,447.483,9602.987,...,0.242,0.0,36248.937,1.124,0.890,38411.949,1.082,30.153333,1,S1VRC0706-2_Excel.txt
2,110.975,145.422,106.843,121.155,115.200,108.267,715615.786,9672.618,456.857,10234.174,...,0.298,0.0,38690.474,1.134,0.882,40936.697,1.064,30.153333,1,S1VRC0706-2_Excel.txt
3,118.437,145.725,106.540,156.515,155.962,100.055,869892.132,11017.111,457.809,12532.360,...,0.216,0.0,44068.445,1.469,0.681,50129.440,1.559,30.153333,1,S1VRC0706-2_Excel.txt
4,115.164,149.916,98.131,160.663,164.922,88.726,799748.054,10416.606,470.976,11947.252,...,0.226,0.0,41666.426,1.637,0.611,47789.007,1.859,30.153333,1,S1VRC0706-2_Excel.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79930,2.811,3.855,3.160,4.135,3.615,2.428,11.626,6.205,12.112,8.320,...,0.441,0.0,24.819,1.309,0.764,33.281,1.489,0.000000,0,MW_I_718_0-10.txt
79931,2.766,3.879,3.294,4.007,3.652,2.485,11.077,6.008,12.185,8.407,...,0.440,0.0,24.031,1.217,0.822,33.627,1.470,0.000000,0,MW_I_718_0-10.txt
79932,2.737,3.739,3.138,3.956,3.511,2.440,10.732,5.882,11.746,7.898,...,0.447,0.0,23.529,1.261,0.793,31.594,1.439,0.000000,0,MW_I_718_0-10.txt
79933,2.679,3.641,3.147,3.840,3.332,2.423,10.072,5.638,11.440,7.423,...,0.450,0.0,22.554,1.220,0.820,29.692,1.375,0.000000,0,MW_I_718_0-10.txt


In [314]:
classData = data
regdata = flow
testData = data
trymodel(train.drop(columns=["Powder"]), flow.drop(columns=["Powder"]), test.drop(columns=["Powder"]))

(271277, 28)
8478/8478 [==============================] - 38s 4ms/step - loss: 0.5847 - accuracy: 0.7076
Model: "sequential_178"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_315 (Conv1D)         (None, 27, 32)            128       
                                                                 
 batch_normalization_293 (Ba  (None, 27, 32)           128       
 tchNormalization)                                               
                                                                 
 activation_213 (Activation)  (None, 27, 32)           0         
                                                                 
 max_pooling1d_306 (MaxPooli  (None, 13, 32)           0         
 ng1D)                                                           
                                                                 
 conv1d_316 (Conv1D)         (None, 13, 64)            6208      
             